In [12]:
import datetime as dt
import geopandas as gpd
from owslib.ogcapi.features import Features

# OGC endpoint hosting near real time fire data
OGC_URL = "https://firenrt.delta-backend.com"

# Initialize an OGC API Features client
w = Features(url=OGC_URL)

# Example near real time (NRT) collection for new fire pixels
# (active fire points) derived from VIIRS
# Options include:
#   "public.eis_fire_snapshot_newfirepix_nrt" (most recent 20 days)
#   "public.eis_fire_lf_newfirepix_nrt"       (current year to date)
COLLECTION = "public.eis_fire_snapshot_newfirepix_nrt"
#COLLECTION = "public.eis_fire_lf_newfirepix_nrt"  

# Determine the most recent time from that collection's metadata
info = w.collection(COLLECTION)
time_intervals = info["extent"]["temporal"]["interval"]
# The last interval is [startTime, endTime]
# We'll parse the endTime (most recent)
end_time_str = time_intervals[-1][1]  # e.g. '2024-08-11T00:00:00+00:00'
end_time = dt.datetime.strptime(end_time_str, "%Y-%m-%dT%H:%M:%S+00:00")

# Choose a start time (e.g. 7 days before end_time) to fetch near real time data
start_time = end_time - dt.timedelta(days=3)
start_time_str = start_time.strftime("%Y-%m-%dT%H:%M:%S+00:00")

# Construct a query to fetch features within the date range
# You can optionally specify a bbox or a filter. We'll just do time range for demonstration.
items = w.collection_items(
    COLLECTION,
    limit=9999,  # Raise if you expect more than 2k features
    datetime=f"{start_time_str}/{end_time_str}"
)

# Convert to a GeoDataFrame
fire_pixels = gpd.GeoDataFrame.from_features(items["features"])
fire_pixels = fire_pixels.set_crs("epsg:4326")  # Confirm correct CRS

# Save to local files as needed
fire_pixels.to_file("active_fire_points_nrt2.geojson", driver="GeoJSON")
fire_pixels.to_file("active_fire_points_nrt2.shp")  # Shapefile example

print("Downloaded near real time active fire points from the OGC API.")


Downloaded near real time active fire points from the OGC API.


In [10]:
import datetime as dt
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from owslib.ogcapi.features import Features

# OGC endpoint hosting near real time fire data
OGC_URL = "https://firenrt.delta-backend.com"

# Initialize an OGC API Features client
w = Features(url=OGC_URL)

# Example near real time (NRT) collection for new fire pixels (active fire points)
COLLECTION = "public.eis_fire_lf_newfirepix_nrt"

# Determine the most recent time from that collection's metadata
info = w.collection(COLLECTION)
time_intervals = info["extent"]["temporal"]["interval"]
# The last interval is [startTime, endTime]
end_time_str = time_intervals[-1][1]  # e.g. '2024-08-11T00:00:00+00:00'
end_time = dt.datetime.strptime(end_time_str, "%Y-%m-%dT%H:%M:%S+00:00")

# Choose a start time (e.g. 3 days before end_time) to fetch near real time data
start_time = end_time - dt.timedelta(days=7)
start_time_str = start_time.strftime("%Y-%m-%dT%H:%M:%S+00:00")

# Construct a query to fetch features within the date range
items = w.collection_items(
    COLLECTION,
    limit=9999,  # Increase as needed
    datetime=f"{start_time_str}/{end_time_str}"
)

# Convert to a GeoDataFrame
fire_pixels = gpd.GeoDataFrame.from_features(items["features"])
fire_pixels = fire_pixels.set_crs("EPSG:4326")  # Confirm correct CRS

# --- FIX FOR MIXED SINGLE/MULTI GEOMETRIES ---
# Some records may have POINT geometries and others MULTIPOINT.
# Shapefiles can't handle a mix. Let's ensure they're all simple Points.
def ensure_point(geom):
    # If it's a Point, keep it
    if isinstance(geom, Point):
        return geom
    # If it's a MultiPoint with at least one geometry, take the first (or merge)
    elif isinstance(geom, MultiPoint) and len(geom.geoms) > 0:
        return geom.geoms[0]
    # Otherwise, skip (or handle differently)
    return None

fire_pixels["geometry"] = fire_pixels["geometry"].apply(ensure_point)
# Drop rows without valid geometry
fire_pixels = fire_pixels[fire_pixels["geometry"].notna()].copy()

# Now all geometry is guaranteed to be Point, so we can safely save a shapefile
fire_pixels.to_file("active_fire_points_nrt2.geojson", driver="GeoJSON")
fire_pixels.to_file("active_fire_points_nrt2.shp")

print("Downloaded and saved near real time active fire points (as POINT geometries).")


Downloaded and saved near real time active fire points (as POINT geometries).
